---
title: Part 2. Introduction to Actinia
description: Learn how to use actinia to create a cloud-based processing environment for GRASS GIS.
format: html
author:
    - Corey T. White
    - Vaclav Petras
date: '2024-09-10'
keep-ipynb: true
toc: true
toc-depth: 4
image:  images/webinar_title.webp
categories: [geospatial, GRASS v8.5, jupyter, lidar, STAC]
page-layout: full
title-block-banner: true
---

In [58]:
!figlet "actinia 101"

            _   _       _         _  ___  _ 
  __ _  ___| |_(_)_ __ (_) __ _  / |/ _ \/ |
 / _` |/ __| __| | '_ \| |/ _` | | | | | | |
| (_| | (__| |_| | | | | | (_| | | | |_| | |
 \__,_|\___|\__|_|_| |_|_|\__,_| |_|\___/|_|
                                            


## A REST API for GRASS GIS

- [GRASS GIS](https://grass.osgeo.org/) is a Geospatial Processing Engine
- Open Source (GPL v2)
- Developed by International and Multi-institutional groups and individuals (GRASS Development Team)
- Member of the Open Source Geospatial Foundatispace
- Recieved Open Source Security Foundation (OpenSSF) Best Practices Badge - 2024

```{mermaid}
flowchart LR
  A[Client] -->|Http Request| B[Actinia REST API]
  B <--> D[GRASS GIS Processing Node]
  B -->|Http Response| A
```


## Import Python Libraries

In [2]:
import os
import subprocess
from pprint import pprint
import sys
import json
import time

import requests
from requests.auth import HTTPBasicAuth

## Setup Environment

In [32]:
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
GISBASE = gisbase
ACTINIA_USER = 'actinia-gdi'
ACTINIA_PASSWORD = 'actinia-gdi'
AUTH = 'actinia-gdi:actinia-gdi'
ACTINIA_URL = 'http://localhost:8088'
# ACTINIA_URL = 'https://openplains.app/actinia'
ACTINIA_VERSION = 'v3'
ACTINIA_BASEURL = 'http://localhost:8088'
# ACTINIA_BASEURL = 'https://openplains.app/actinia'
ACTINIA_URL = ACTINIA_BASEURL + "/api/" + ACTINIA_VERSION
ACTINIA_AUTH = HTTPBasicAuth(ACTINIA_USER, ACTINIA_PASSWORD)

## actinia REST API

[actinia](https://github.com/actinia-org)
[Docker Image](https://github.com/actinia-org/actinia-docker/tree/main)

### Tools

- [actinia-python-client](https://github.com/actinia-org/actinia-python-client)
- [actinia_openapi_python_client](https://github.com/OpenPlainsInc/actinia_openapi_python_client)

In [ ]:
!pip3 install actinia-python-client
!pip3 install git+https://github.com/openplainsinc/actinia_openapi_python_client.git@v0.0.4

## Acinia Basics

In [59]:
url = f"{ACTINIA_URL}/version"
response = requests.get( url)
# pprint(response.json())
pprint(response.json().get('version'))

'4.14.1'


## User Roles

1. **Superadmin**
    - Create, modify and delete users
    - Read/write access to all databases
    - Can access all API calls

2. **Admin**
    - All API Calls
    - Create, modify and delete users with the maximum user-role user of the same user grou
    - Access to persistent databases that were granted by a **superadmin**

3. **User**
    - Can run computational tasks in ephemeral and user specific databases
    - Create, modify and delete locations in a user specific database
    - Create, modify and delete mapsets in user specific databases
    - Limited access to API calls
    - Has limited access to persistent databases

4. **Guest**
    - Has very limited access to API calls
    - Limited access to persistent databases

:::aside
https://actinia-org.github.io/actinia-core/actinia_concepts/
:::


### GRASS Projects

Let's start by requesting a list of GRASS projects from actinia.

```{mermaid}
flowchart LR
  A["/api/v3/locations"]
```

In [62]:
url = f"{ACTINIA_URL}/locations"

response = requests.get( url, auth=ACTINIA_AUTH)

pprint(response.json().get('locations'))


['nc_spm_08']


Now we will get extra information about a specific project.

```{mermaid}
flowchart LR
  A["/api/v3/locations/"]
  A --- C["{location_id}"]
  C --- D["/info"]
```

In [17]:
location_id = 'nc_spm_08'
url = f"{ACTINIA_URL}/locations/{location_id}/info"

payload = {}
headers = {}

response = requests.get( url, auth=ACTINIA_AUTH, headers=headers)

pprint(response.json().get('process_results'))

{'projection': 'PROJCRS["NAD83(HARN) / North '
               'Carolina",BASEGEOGCRS["NAD83(HARN)",DATUM["NAD83 (High '
               'Accuracy Reference Network)",ELLIPSOID["GRS '
               '1980",6378137,298.257222101,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4152]],CONVERSION["SPCS83 '
               'North Carolina zone (meters)",METHOD["Lambert Conic Conformal '
               '(2SP)",ID["EPSG",9802]],PARAMETER["Latitude of false '
               'origin",33.75,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8821]],PARAMETER["Longitude '
               'of false '
               'origin",-79,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8822]],PARAMETER["Latitude '
               'of 1st standard '
               'parallel",36.1666666666667,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8823]],PARAMETER["Latitude '
               'of 2nd standard '
               'parallel",34.3333333333333,ANGLEUNIT["degree",0.01745

### Mapsets

We will now get a list of mapsets for a specific project.

```{mermaid}
flowchart LR
  A["/api/v3/locations/"]
  A --- B("{location_id}")
  B --- C["/mapsets"]
```

In [22]:
# mapset = 'nc_spm_08'
location_id = 'nc_spm_08'
url = f"{ACTINIA_URL}/locations/{location_id}/mapsets"

payload = {}
headers = {}

response = requests.get( url, auth=ACTINIA_AUTH, headers=headers)

pprint(response.json().get('process_results'))

['PERMANENT']


Now let's get extra information about a specific mapset `PERMANENT`.

```{mermaid}
flowchart LR
  A["/api/v3/locations/"]
  A --- B("{location_id}")
  B --- C["/mapsets/"]
  C --- D("{mapset_id}")
  D --- E["/info"]
```

In [63]:
mapset_id = 'PERMANENT'
location_id = 'nc_spm_08'
url = f"{ACTINIA_URL}/locations/{location_id}/mapsets/{mapset_id}/info"

response = requests.get( url, auth=ACTINIA_AUTH, headers=headers)

pprint(response.json().get('process_results'))

{'projection': 'PROJCRS["NAD83(HARN) / North '
               'Carolina",BASEGEOGCRS["NAD83(HARN)",DATUM["NAD83 (High '
               'Accuracy Reference Network)",ELLIPSOID["GRS '
               '1980",6378137,298.257222101,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4152]],CONVERSION["SPCS83 '
               'North Carolina zone (meter)",METHOD["Lambert Conic Conformal '
               '(2SP)",ID["EPSG",9802]],PARAMETER["Latitude of false '
               'origin",33.75,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8821]],PARAMETER["Longitude '
               'of false '
               'origin",-79,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8822]],PARAMETER["Latitude '
               'of 1st standard '
               'parallel",36.1666666666667,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8823]],PARAMETER["Latitude '
               'of 2nd standard '
               'parallel",34.3333333333333,ANGLEUNIT["degree",0.017453

### Layers

#### Raster Layers

Let's see all of the mapsets raster layers.

```{mermaid}
flowchart LR
  A["/api/v3/locations/"]
  A --- B("{location_id}")
  B --- C["/mapsets/"]
  C --- D("{mapset_id}")
  D --- E["/raster_layers"]
```

In [21]:
mapset_id = 'PERMANENT'
location_id = 'nc_spm_08'
url = f"{ACTINIA_URL}/locations/{location_id}/mapsets/{mapset_id}/raster_layers"

payload = {}
headers = {}

response = requests.get( url, auth=ACTINIA_AUTH, headers=headers)

pprint(response.json().get('process_results'))

[]


#### Vector Layers

```{mermaid}
flowchart LR
  A["/api/v3/locations/"]
  A --- B("{location_id}")
  B --- C["/mapsets/"]
  C --- D("{mapset_id}")
  D --- E["/vector_layers"]
```

In [20]:
mapset_id = 'PERMANENT'
location_id = 'nc_spm_08'
url = f"{ACTINIA_URL}/locations/{location_id}/mapsets/{mapset_id}/vector_layers"

payload = {}
headers = {}

response = requests.get( url, auth=ACTINIA_AUTH, headers=headers)

pprint(response.json().get('process_results'))

[]


### Processing

#### Syncronous Processing

Emphemeral process

```{mermaid}
flowchart LR
  A["/api/v3/locations/"]
  A --- B("{location_id}")
  B --- C["/mapsets/"]
  C --- D("{mapset_id}")
  D --- E["/processing"]
```

In [37]:
mapset_id = 'PERMANENT'
location_id = 'nc_spm_08'
url = f"{ACTINIA_URL}/locations/{location_id}/mapsets/{mapset_id}/processing"

payload = {}
headers = {}
data = {}

response = requests.post( url, auth=ACTINIA_AUTH, json=data)

pprint(response.json().get('process_results'))

None


#### Asyncronous Processing

```{mermaid}
flowchart LR
  A["/api/v3/locations/"]
  A --- B("{location_id}")
  B --- C["/mapsets/"]
  C --- D("{mapset_id}")
  D --- E["/processing_async"]
```

In [ ]:
mapset_id = 'PERMANENT'
location_id = 'nc_spm_08'
url = f"{ACTINIA_URL}/locations/{location_id}/mapsets/{mapset_id}/processing_async"

payload = {}
headers = {}
data = {}

response = requests.post( url, auth=ACTINIA_AUTH, headers=headers, json=data)

pprint(response.json().get('process_results'))

### Process Chains

Let's build our own process chain.

In [ ]:
pc = {
    "list": [
        {
             "id": "computational_region",
             "module": "g.region",
             "inputs": [
                 {"param": "raster",
                  "value": "elevation@PERMANENT"},
                 {"param": "res",
                  "value": "10"}
             ],
             "stdout": {"id": "region", "format": "kv", "delimiter": "="},
             "flags": "g"
         },
        {
          "id": "create_hillshading",
          "module": "r.relief",
          "inputs": [
              {
                  "param": "input",
                  "value": "elevation"
              }
          ],
          "outputs": [
              {
                  "param": "output",
                  "value": "hillshade"
              }
          ]
      },
      {
          "id": "exporter_1",
          "module": "exporter",
          "outputs": [
              {
                  "export": {"type": "raster", "format": "COG"},
                  "param": "map",
                  "value": "hillshade"
              }
          ]
      }
    ],
    "version": "1"
}

In [45]:
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)
# create a temporary folder where to place our GRASS project
import tempfile
from pathlib import Path

tempdir = tempfile.TemporaryDirectory()

import grass.script as gs
import grass.jupyter as gj

gs.create_project(path=tempdir.name, name="foss4g2024_p2", epsg="2817", overwrite=True)
session = gj.init(Path(tempdir.name,"foss4g2024_p2"))

In [57]:
step_1_raw = !g.region raster=elevation res=10 --json
step_1 = ''.join(step_1_raw)
step_1_json = json.loads(step_1)
pprint(step_1_json)

{'id': 'g.region_1804289383',
 'inputs': [{'param': 'raster', 'value': 'elevation'},
            {'param': 'res', 'value': '10'},
            {'param': 'format', 'value': 'plain'}],
 'module': 'g.region'}


In [47]:
!r.info elevation -g --json

{
  "module": "r.info",
  "id": "r.info_1804289383",
  "flags":"g",
  "inputs":[
     {"param": "map", "value": "elevation"},
     {"param": "format", "value": "plain"}
   ]}
